In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x, y, t):
        input_data = torch.stack([x.view(-1), y.view(-1), t.view(-1)], dim=1)
        #input_data = torch.cat((x, y, t), dim=-1)
        return self.model(input_data)

def linear_advection_ic(x, y):
    return np.sin(2 * np.pi * (x)) #+ np.sin(2 * np.pi * y)

def linear_advection_bc(x, y, t):
    exact = linear_advection_ic(x-t, y-t)
    return exact

def linear_advection(u, x, y,t):
    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    return u_t + 1*u_x + 1*u_y

def linear_advection_exact_solution(x, y, t):
    return linear_advection_ic(x - t, y - t)


def train_PINN(model, x_initial, y_initial, t_initial, x_boundary, y_boundary, t_boundary, x_collocation, y_collocation, t_collocation, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad() 

        x_initial_grid, y_initial_grid = np.meshgrid(x_initial, y_initial)
        t_initial_grid = np.full_like(x_initial_grid, t_initial)

        x_initial_tensor, y_initial_tensor, t_initial_tensor, x_boundary_tensor, y_boundary_tensor, t_boundary_tensor = (
            torch.tensor(x, dtype=torch.float32, device = "cuda", requires_grad=True) for x in
            [x_initial_grid, y_initial_grid, t_initial_grid, x_boundary, y_boundary, t_boundary]
        )


        u_initial_pred, u_boundary_pred1, u_boundary_pred2, u_boundary_pred3, u_boundary_pred4 = (
            model(x, y, t) for x, y, t in
            [(x_initial_tensor, y_initial_tensor, t_initial_tensor), 
            (x_boundary_tensor, torch.zeros_like(y_boundary_tensor), t_boundary_tensor), 
            (torch.ones_like(x_boundary_tensor), y_boundary_tensor, t_boundary_tensor),
            (x_boundary_tensor, torch.ones_like(y_boundary_tensor), t_boundary_tensor),
            (torch.zeros_like(x_boundary_tensor), y_boundary_tensor, t_boundary_tensor)]
        )

        
        u_initial_exact, u_boundary_exact1, u_boundary_exact2, u_boundary_exact3, u_boundary_exact4 = (
            torch.tensor(u, device = "cuda", dtype=torch.float32) for u in [linear_advection_ic(x_initial_grid, y_initial_grid), 
            linear_advection_bc(x_boundary, np.zeros_like(y_boundary), t_boundary),
            linear_advection_bc(np.ones_like(x_boundary), y_boundary, t_boundary),
            linear_advection_bc(x_boundary, np.ones_like(y_boundary), t_boundary),
            linear_advection_bc(np.zeros_like(x_boundary), y_boundary, t_boundary)]
        )
        
        loss1 = loss_fn(u_initial_pred, u_initial_exact.view(-1,1))
        loss2 = loss_fn(u_boundary_pred1, u_boundary_exact1)
        loss3 = loss_fn(u_boundary_pred2, u_boundary_exact2)
        loss4 = loss_fn(u_boundary_pred3, u_boundary_exact3)
        loss5 = loss_fn(u_boundary_pred4, u_boundary_exact4)
        

        loss6 = 0
        for t_collocation_batch_ in t_collocation:

            # 랜덤한 Collocation points 생성
            idx = np.random.choice(len(x_collocation), size=50, replace=False)

            x_collocation_batch = x_collocation[idx]
            y_collocation_batch = y_collocation[idx]
            t_collocation_batch = np.full(x_collocation_batch.shape, t_collocation_batch_)

            
            x_collocation_tensor, y_collocation_tensor, t_collocation_tensor = (
                torch.tensor(x_collocation_batch, dtype=torch.float32,device = "cuda", requires_grad=True) for x_collocation_batch in
                [x_collocation_batch, y_collocation_batch, t_collocation_batch]
            )

            u_collocation_pred = model(x_collocation_tensor, y_collocation_tensor, t_collocation_tensor)

            equation_residual = linear_advection(u_collocation_pred, x_collocation_tensor, y_collocation_tensor, t_collocation_tensor)
            loss6 += loss_fn(equation_residual, torch.zeros_like(equation_residual))

        loss = loss1 + loss2 + loss3 + loss4 + loss5 + loss6
        
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(loss1.item(), loss2.item(), loss3.item(), loss4.item(), loss5.item(), loss6.item())
            print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item()}")

    print("Training completed.")

# 학습 데이터 생성 (2D 선형 이동 방정식)
x_initial = np.linspace(0, 1, 100).reshape(-1, 1)
y_initial = np.linspace(0, 1, 100).reshape(-1, 1)
t_initial = np.zeros_like(x_initial)
x_boundary = np.linspace(0, 1, 100).reshape(-1, 1)
y_boundary = np.linspace(0, 1, 100).reshape(-1, 1)
t_boundary = np.linspace(0, 2, 100).reshape(-1, 1)
x_collocation = np.random.uniform(0, 1, size=(100, 1))
y_collocation = np.random.uniform(0, 1, size=(100, 1))
t_collocation = np.linspace(0,1,50).reshape(-1,1)
# 모델 생성 및 학습 (2D 선형 이동 방정식)
model = PINN().cuda()
train_PINN(model, x_initial, y_initial, t_initial, x_boundary, y_boundary, t_boundary, x_collocation, y_collocation, t_collocation, epochs=500, learning_rate=0.01)

In [ ]:
# x, y 그리드 생성
x_grid = np.linspace(0, 1, 100)
y_grid = np.linspace(0, 1, 100)
x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)

t_0 = 0
t_2 = 2.0

# 예측된 그래프와 정확한 그래프 계산
u_exact_0 = linear_advection_exact_solution(x_mesh, y_mesh, t_0)
u_exact_2 = linear_advection_exact_solution(x_mesh, y_mesh, t_2)

# t=0초와 t=2초에서의 u(x, y) 계산
t_0 = np.array([t_0 for i in range(len(x_grid)*len(y_grid))])
t_2 = np.array([t_2 for i in range(len(x_grid)*len(y_grid))])

# 모델을 사용하여 예측
u_0 = model(torch.tensor(x_mesh, device = "cuda", dtype=torch.float32).view(-1, 1),
            torch.tensor(y_mesh, device = "cuda", dtype=torch.float32).view(-1, 1),
            torch.tensor(t_0, device = "cuda", dtype=torch.float32).view(-1, 1)).cpu().detach().numpy()

u_2 = model(torch.tensor(x_mesh, device = "cuda", dtype=torch.float32).view(-1, 1),
            torch.tensor(y_mesh, device = "cuda", dtype=torch.float32).view(-1, 1),
            torch.tensor(t_2, device = "cuda", dtype=torch.float32).view(-1, 1)).cpu().detach().numpy()


# 2D 그래프 그리기
plt.figure(figsize=(12, 6))

# t=0초에서의 그래프 비교
plt.subplot(121)
plt.pcolormesh(x_grid, y_grid, u_0.reshape(100, 100), cmap='viridis')
plt.colorbar(label='u(x, y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('u(x, y) at t=0 sec (Predicted)')

plt.subplot(122)
plt.pcolormesh(x_grid, y_grid, u_exact_0.reshape(100, 100), cmap='viridis')
plt.colorbar(label='u(x, y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('u(x, y) at t=0 sec (Exact)')

plt.tight_layout()
plt.show()

# t=2초에서의 그래프 비교
plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.pcolormesh(x_grid, y_grid, u_2.reshape(100, 100), cmap='viridis')
plt.colorbar(label='u(x, y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('u(x, y) at t=2 sec (Predicted)')

plt.subplot(122)
plt.pcolormesh(x_grid, y_grid, u_exact_2.reshape(100, 100), cmap='viridis')
plt.colorbar(label='u(x, y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('u(x, y) at t=2 sec (Exact)')

plt.tight_layout()
plt.show()